# Project 2:
## Authors: Frederick Nilsen, Mia Rødde, Sara Abnar

### Warnings

* Ikke endre uten at de andre vet det
* **Alltid** gå på Kernel -> Restart & Clear Output
* Ikke skriv utenfor cellene og sånn

## Implementation of set 1 - points in 2D

In [ ]:
## IMPORTS

import numpy as np
import matplotlib.pyplot as plt
from loader import get_dataset
import plotting
from spirals import get_data_spiral_2d

In [ ]:
## GLOBAL CONSTS AND DEFINITIONS

sigma = lambda x : np.tanh(x)         # Activation function
eta = lambda x : 0.5*(1+np.tanh(x/2)) # Scalar function     
eta_ddx = lambda x :  0.25*(1-np.tanh(x/2)*np.tanh(x/2)) # Derivative of eta
sigma_ddx = lambda x: 1-(np.tanh(x))**2 # Derivative of sigma


K = 10 # +1 gives number of layers, arbitrary number
d = 2 # Ehh
tau = 0.1 # learning parameter [0.01,0.1]
Y0, C = get_data_spiral_2d(200) #Default n=200
I = Y0.shape[1]

np.random.seed(0)
W = np.random.randn(K,d,d)
np.random.seed(0)
b = np.random.randn(K,d,1)     #startverdier
np.random.seed(0)
mu = np.random.randn()
np.random.seed(0)
w = np.random.randn(d,1)
h = 0.1

Ulst = [W, b, w, mu]

In [ ]:
## INITIALISATION CELL
def getColorFromLabel(C):
    colorLst = []
    for el in C:
        if el:
            colorLst.append('r')
        else:
            colorLst.append('b')
    return colorLst
C_colors = getColorFromLabel(C)


#optimert funksjon
def YK(W, b, Ystart=Y0):
    #tom Y-matrise, 3-dimensjonalt numpy-array
    Ym = np.zeros((K+1,d,I))
    Ym[0,:,:] = np.copy(Ystart)
    k=0
    while k < K:
        Ym[k+1] = Ym[k] + h*sigma(W[k]@Ym[k] + b[k])
        k+=1
    return Ym

#Finner P_K
def getGradients(K, W, b, w, mu, c=C, Ystart=Y0):
    Y_all = YK(W,b, Ystart)
    lastY = Y_all[-1,:,:]
    P = np.zeros((Y_all.shape)) # init matrix
    P[K,:,:] = w@np.transpose((Zf(w,mu, lastY)-c)*eta_ddx(np.transpose(lastY)@w+mu))  #(7)
    
    dJdMU = eta_ddx(np.transpose(np.transpose(lastY)@w+mu))@(Zf(w,mu, lastY)-c)   #(5), kunne også brukt '.T' for transpose
    dJdw = lastY@((Zf(w,mu, lastY)-c)*eta_ddx(np.transpose(lastY)@w+mu))              #(6)
    for k in range(K,0,-1):
        P[k-1,:,:] = P[k,:,:]+h*np.transpose(W[k-1,:,:])@(sigma_ddx(W[k-1,:,:]@Y_all[k-1,:,:]+b[k-1,:,:])*P[k,:,:])  #(8) 
    dJdWk = np.zeros((K,d,d))
    dJdBk = np.zeros((K,d,1))
    
    for k in range(0,K):
        dJdWk[k] = h*(P[k+1]*sigma_ddx(W[k,:,:]@Y_all[k,:,:]+b[k,:,:]))@np.transpose(Y_all[k,:,:])  #(9)
        dJdBk[k] = h*(P[k+1]*sigma_ddx(W[k,:,:]@Y_all[k,:,:]+b[k,:,:]))@np.ones((I,1)) #One not needed?#(10)
    return dJdWk, dJdBk, dJdw, dJdMU


#Adam descent algorithm
def AdamAlg(uParam, U_ind, lastM, lastV, c=C, Ystart=Y0):
    beta1 = 0.9
    beta2 = 0.999
    alpha = 0.01
    epsilon = 1e-8
    g = getGradients(K, W, b, w, mu, c, Ystart)[U_ind]
    m = beta1*lastM+(1-beta1)*g
    v = beta2*lastV+(1-beta2)*(g*g)
    mhat = m/(1-beta1**(counter))
    vhat = v/(1-beta2**(counter))
    uParam -= alpha*mhat/(np.sqrt(vhat)+epsilon)
    lastM = m
    lastV = v
    return uParam, lastM, lastV

In [ ]:
#Lager siste Y_k = Y_K
YK_list = YK(W,b)
lastY = YK_list[-1,:,:]

Zf = lambda w,mu,finalY=lastY : eta(np.transpose(finalY)@w +mu)
Z=Zf(w,mu)
Jf = lambda Z=Zf(w,mu), ce=C : 1/2*np.linalg.norm(Z-ce)**2 #Cost function
J = Jf()

In [ ]:
#INITIALISING LIST, PART 2
pic, C_pic = get_dataset()
K = 10 #number of layers, arbitrary number
d = pic.shape[0]
I = pic.shape[1]

np.random.seed(0)
W = np.random.randn(K,d,d)
np.random.seed(0)
b = np.random.randn(K,d,1)     #startverdier
np.random.seed(0)
mu = np.random.randn()
np.random.seed(0)
w = np.random.randn(d,1)
h = 0.1

In [ ]:
Y0pic, C_pic = get_dataset()
d = Y0pic.shape[0]
I = Y0pic.shape[1]


#algorithm for pictures
counter = 1

#initializing list
thisMWpic = thisVWpic = 0
thisMbpic = thisVbpic = 0
thisMwpic = thisVwpic = 0
thisMmupic = thisVmupic = 0


#for AdamAlg
for i in range(1,11):   #Skal være 40001
    W, thisMWpic, thisVWpic = AdamAlg(W, 0, thisMWpic, thisVWpic, C_pic, Y0pic)
    b, thisMbpic, thisVbpic = AdamAlg(b, 1, thisMbpic, thisVbpic, C_pic, Y0pic)
    w, thisMwpic, thisVwpic = AdamAlg(w, 2, thisMwpic, thisVwpic, C_pic, Y0pic)
    mu, thisMmupic, thisVmupic = AdamAlg(mu, 3, thisMmupic, thisVmupic, C_pic, Y0pic)
    counter +=1 
    
    
testpic, testCp = get_dataset()
trainedPic = YK(W, b, testpic)
lastY = trainedPic[-1,:,:]
print(Jf(Zf(w, mu, lastY), testCp))